# Value Function Approximation

From table to function.

## TD learning of state values based on function approximation

Let $v_{\pi}(s)$ and $\hat{v}(s, w)$ be the true state value and approximated state value of $s\in\mathcal{S}$, respectively. The problem to be solved is to find an optimal $w$ so that $\hat{v}(s, w)$ can best approximate $v_{\pi}(s)$ for every $s$. In particular, the objective function is

$$J(w) = \mathbb{E}[(v_{\pi}(S) - \hat{v}(S, w))^{2}]$$

where the expectation is calculated with respect to the random variable $S\in\mathcal{S}$. There are several ways to define the probability distribution.

* Uniform distribution:

$$J(w) = \frac{1}{n}\sum_{s\in\mathcal{S}}(v_{\pi}(s) - \hat{v}(s, w))^{2}$$

* Use the stationary distribution. After the agent executes a given policy for a sufficiently long period, the probability of the agent being located at any state can be described by this stationary distribution $\{d_{\pi}(s)\}_{s\in\mathcal{S}}$

$$J(w) = \sum_{s\in\mathcal{S}}d_{\pi}(s)(v_{\pi}(s) - \hat{v}(s, w))^{2}$$

To minimize the objective function, we can use gradient descent:

$$
\begin{aligned}
w_{k+1} &= w_{k} - \alpha_{k}\nabla_{w}J(w_{k})\\
&= w_{k} + 2\alpha_{k}\mathbb{E}[(v_{\pi}(S) - \hat{v}(S, w_{k}))\nabla_{w}\hat{v}(S, w_{k})]
\end{aligned}
$$

Stochastic gradient descent:

$$w_{t+1} = w_{t} + \alpha_{t}(v_{\pi}(s_{t}) - \hat{v}(s_{t}, w_{t}))\nabla_{w}\hat{v}(s_{t}, w_{t})$$

We can replace $v_{\pi}(s_{t})$ with approximation to make this algorithm implementable:

* Use Monte Carlo method to approximate state values.
* Temporal difference method:
$$w_{t+1} = w_{t} + \alpha_{t}\left[r_{t+1} + \gamma\hat{v}(s_{t+1}, w_{t}) - \hat{v}(s_{t}, w_{t})\right]\nabla_{w}\hat{v}(s_{t}, w_{t})$$

## Sarsa with function approximation

The Sarsa algorithm with function approximation can be readily obtained by replacing the state values with action values. Suppose $q_{\pi}(s, q)$ is approximated by $\hat{q}(s, a, w)$

$$w_{t+1} = w_{t} + \alpha_{t}\left[ r_{t+1} + \gamma\hat{q}(s_{t+1}, a_{t+1}, w_{t}) - \hat{q}(s_{t}, a_{t}, w_{t})\right]\nabla_{w}\hat{q}(s_{t}, a_{t}, w_{t})$$

## Q-learning with function approximation

Tabular Q-learning can also be extended to the case of function approximation:

$$w_{t+1} = w_{t} + \alpha_{t}\left[ r_{t+1} + \gamma\underset{a\in\mathcal{A}}{\max}\hat{q}(s_{t+1}, a, w_{t}) - \hat{q}(s_{t}, a_{t}, w_{t})\right]\nabla_{w}\hat{q}(s_{t}, a_{t}, w_{t})$$

## Deep Q-learning

Deep Q-learning aims to minimize the following objective function:

$$J(w) = \mathbb{E}\left[\left(R + \gamma\underset{a\in\mathcal{A}}{\max}\hat{q}(S', a, w) - \hat{q}(S,A,w)\right)^{2}\right]$$

Note that the parameter $w$ appears not only in $\hat{q}(S,A,w)$ but also in $y=R + \gamma\underset{a\in\mathcal{A}}{\max}\hat{q}(S', a, w)$, it is nontrival to calculate the gradient. We introduce two networks: one is a main network representing $\hat{q}(s,a,w)$ and the other is a target network $\hat{q}(s,a,w_{T})$, the objective function becomes:

$$J(w) = \mathbb{E}\left[\left(R + \gamma\underset{a\in\mathcal{A}}{\max}\hat{q}(S', a, w_{T}) - \hat{q}(S,A,w)\right)^{2}\right]$$

where $w_{T}$ is the target network's parameter. When $w_{T}$ fixed, the gradient of $J$ is

$$\nabla_{w}J(w) = \mathbb{E}\left[\left(R + \gamma\underset{a\in\mathcal{A}}{\max}\hat{q}(S', a, w_{T}) - \hat{q}(S,A,w)\right)\nabla_{w}\hat{q}(S,A,w)\right]$$

where some constant coeffients are omitted.

We need to pay attention to the following techniques:

* We use two networks, a main network and a target network. In every iteration, we draw a mini-batch of samples $\{(s,a,r,s')\}$ from the replay buffer, target output $y_{T}=r+ \gamma\underset{a\in\mathcal{A}}{\max}\hat{q}(s',a,w_{T})$, the main network is updated to minimize the TD error over samples $\{(s,a,y_{T})\}$. The main network is updated in every iteration, the target network is set to be the same as the main network every certain number of iterations.

* Experience replay, after we collected some experience samples, we do not use these samples in the order they were collected, instead, we store them in a dataset called replay buffer, store most recent $n$ samples. Every time we update the main network, we can draw a mini-batch of samples uniformly from the replay buffer. By using experience replay, we can avoid waste of experience and correlated updates.